# Building a Deployment Environment
Create an S3 bucket for the build, deploy the Lambda binaries and CloudFormation template, and run CloudFormation.

# [Required] 
For running Cloud Formation, some roles below are needed:  
- AmazonS3FullAccess or ListBucket/GetObject/DeleteObject/PutObject  
- AWSLambdaFullAccess  
- IAMFullAccess  
- AWSCloudTrailFullAccess  
- CloudWatchLogsFullAccess  
- AWSStepFunctionsFullAccess  
- AmazonSageMakerFullAccess  
- AWSCloudFormationFullAccess  

## Create S3 bucket  
Create an S3 bucket to store the Lambda binary file and the Cloud Formation template file to be used for environment preparation.

In [ ]:
import boto3

In [ ]:
session = boto3.Session()
region = session.region_name
account_id = session.client('sts').get_caller_identity().get('Account')
bucket_name = f'workshop-timeseries-retail-{account_id}-deploy'
s3 = boto3.client('s3')

In [ ]:
bucket_name

In [ ]:
s3.create_bucket(Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint':region}
                )

## Put Lambda zip files to S3

In [ ]:
import glob

files = glob.glob("lambdas_deploy/*")
for file in files:
    print(file)
    s3.upload_file(file, bucket_name, file)

## Run CloudFormation

In [ ]:
s3.upload_file('cfn_template.yaml', bucket_name, 'cfn_template.yaml')

In [ ]:
cf = boto3.client('cloudformation')
res = cf.create_stack(
    StackName=('workshop-timeseries-retail'),
    TemplateURL=f'https://workshop-timeseries-retail-{account_id}-deploy.s3.amazonaws.com/cfn_template.yaml',
    Parameters=[
        {
            'ParameterKey': 'BucketName',
            'ParameterValue': bucket_name
        },
        {
            'ParameterKey': 'Account',
            'ParameterValue': account_id
        },
    ],
    Capabilities=[
        'CAPABILITY_IAM',
        'CAPABILITY_AUTO_EXPAND',
    ],
)

## Next
Wait for CloudFormation to complete. When it is complete, switch to the sagemaker notebook instance named workshop-timeseries-retail-ForecastNotebook created by the Cloud Formation and run 1_build_statemachine.ipynb.